<a href="https://colab.research.google.com/github/javier-jaime/Stock_Price_Forecasting/blob/main/Update10_BTC_Price_Forecasting_with_yfinance_and_FB_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bitcoin Price Forecasting with yfinance and Facebook Prophet**

## Importing all the necessary Libraries

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
from prophet import Prophet
import plotly.io as pio
pio.renderers.default='colab'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.6 MB/s eta 0:00:00
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting un

## Importing the Dataset & Exploring it

In [ ]:
 # Stock Ticker Input
stock = input('Enter the stock ticker: ')
print(f'You entered {stock}, here is the data from the last 10 days:')
period = 'max'
ticker = yf.Ticker(stock)
data = ticker.history(period)
data.reset_index(inplace=True)
data.tail(10)

Enter the stock ticker: BTC-USD
You entered BTC-USD, here is the data from the last 10 days:


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
3090,2023-03-04 00:00:00+00:00,22362.923828,22405.177734,22198.980469,22353.349609,11166012913,0.0,0.0
3091,2023-03-05 00:00:00+00:00,22354.144531,22613.685547,22307.142578,22435.513672,13317001733,0.0,0.0
3092,2023-03-06 00:00:00+00:00,22436.816406,22584.292969,22331.314453,22429.757812,17353192895,0.0,0.0
3093,2023-03-07 00:00:00+00:00,22428.322266,22527.417969,22011.261719,22219.769531,22765452204,0.0,0.0
3094,2023-03-08 00:00:00+00:00,22216.441406,22268.896484,21708.050781,21718.080078,22536575684,0.0,0.0
3095,2023-03-09 00:00:00+00:00,21720.080078,21802.716797,20210.306641,20363.021484,30364664171,0.0,0.0
3096,2023-03-10 00:00:00+00:00,20367.001953,20370.595703,19628.253906,20187.244141,39578257695,0.0,0.0
3097,2023-03-11 00:00:00+00:00,20187.876953,20792.525391,20068.660156,20632.410156,30180288176,0.0,0.0
3098,2023-03-12 00:00:00+00:00,20628.029297,22185.031250,20448.806641,22163.949219,29279035521,0.0,0.0
3099,2023-03-13 00:00:00+00:00,22178.580078,24411.455078,21918.199219,24411.455078,50565316608,0.0,0.0


In [ ]:
print('These are the basic stats:')
data.describe()

These are the basic stats:


,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,3100.000000,3100.000000,3100.000000,3100.000000,3.100000e+03,3100.0,3100.0
mean,13063.281745,13389.494539,12701.811925,13069.475761,1.652978e+10,0.0,0.0
std,16060.384806,16469.627510,15588.565036,16056.598895,1.981662e+10,0.0,0.0
min,176.897003,211.731003,171.509995,178.102997,5.914570e+06,0.0,0.0
25%,704.852020,715.353012,697.435013,704.859741,1.101680e+08,0.0,0.0
50%,7295.301758,7447.051025,7152.361816,7299.333496,8.649370e+09,0.0,0.0
75%,18331.917480,18796.491211,17716.345703,18365.591309,2.801814e+10,0.0,0.0
max,67549.734375,68789.625000,66382.062500,67566.828125,3.509679e+11,0.0,0.0


# Data Visualization (plotly express) - Visualizing the historical performance

In [ ]:
#Line graph (Price), Area graph (Volume) , box plot (Analyzing price and volume)

In [ ]:
px.line(data, x="Date", y="Close", title=f'{stock} Stock Close Price')

In [ ]:
px.bar(data, x="Date", y="Volume", title=f'{stock} Stock Volume')

In [ ]:
px.box(data, y="Close", title=f'{stock} box plot', height=500, width=500)

In [ ]:
def get_hurst_exponent(time_series, max_lag=100):
    """Returns the Hurst Exponent of the time series"""
    lags = range(2, max_lag)
    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]
    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)
    return reg[0]

In [ ]:
hurst = pd.DataFrame(columns =["hurst_exp","lag"])
lags = range(4, 61, 1)
for lag in lags:
    hurst_exp = get_hurst_exponent(data["Close"].values, lag)
    hurst = hurst.append({"hurst_exp":hurst_exp,"lag":lag},ignore_index=True)
px.line(hurst, x="hurst_exp", y="lag", title=f'{stock} Hurst Exp')

## Data Preparation for Facebook Prophet

In [ ]:
columns=['Date','Close']
prophet_df=pd.DataFrame(data,columns=columns)
prophet_df= prophet_df.rename(columns={'Date':'ds','Close':'y'})  
prophet_df.tail()

,ds,y
1091,2023-03-09 00:00:00+00:00,20363.021484
1092,2023-03-10 00:00:00+00:00,20187.244141
1093,2023-03-11 00:00:00+00:00,20632.410156
1094,2023-03-12 00:00:00+00:00,22163.949219
1095,2023-03-13 00:00:00+00:00,24274.597656


## Creating Facebook Prophet Model

In [ ]:
m=Prophet(daily_seasonality=False)
m.fit(prophet_df)

ValueError: ignored

##Forecasting

In [ ]:
periods = 30
future=m.make_future_dataframe(periods)

In [ ]:
forecast=m.predict(future)

In [ ]:
forecast.head(10)

In [ ]:
px.line(forecast, x='ds',y='yhat', title=f'{stock} Stock Forecast')

In [ ]:
figure=m.plot(forecast, xlabel='ds', ylabel='y')

In [ ]:
figure2=m.plot_components(forecast)

##Downloading the Forecast data

In [ ]:
from google.colab import files
forecast.to_csv(f'{stock}_forecast.csv')
files.download(f'{stock}_forecast.csv')